In [1]:
# №5 угадывание 5-ого квинтиля (лучших данных) по жанрам и второму рейтингу
import pandas as pd 
import numpy as np
import scipy.stats as sts
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import defaultdict

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import collections
np.random.seed(123)

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 4)
        
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.probability import FreqDist

def popular_word_list(df, n):
    top_words_count = n  # количество самых популярных слов для вывода

    # Создаем массив из 5 списков для хранения самых популярных слов по каждому квинтилю
    quintile_top_words = [[] for _ in range(5)]
    quintile_top_words_list = [[] for _ in range(5)]

    # Обходим каждую строку в таблице df_filtered
    for index, row in df_filtered.iterrows():
        description = row['description'].lower()
        quintile = row['quintile']

        # Токенизируем описание фильма
        tokens = word_tokenize(description)

        # Исключаем незначащие слова
        filtered_tokens = [token for token in tokens if token.lower() not in unmeaning_words]

        # Обновляем частотное распределение слов
        fdist = FreqDist(filtered_tokens)

        # Получаем список самых популярных слов
        top_words = [word for word, _ in fdist.most_common(top_words_count)]

        # Добавляем список самых популярных слов в соответствующий список квинтиля
        quintile_top_words[quintile - 1].extend(top_words)

    for i in range (0, 5):
        # Создаем словарь с подсчетом частоты встречаемости слов
        word_counter = collections.Counter(quintile_top_words[i])

        # Находим 50 самых часто встречающихся слов
        top_words = word_counter.most_common(top_words_count)
        quintile_top_words_list[i] = top_words

    # Создаем новый массив со списками слов без чисел
    words_list = [[pair[0] for pair in sublist] for sublist in quintile_top_words_list]

    merged_list = []
    for sublist in words_list:
        merged_list.extend(sublist)

    unique_words = []
    for word in merged_list:
        if merged_list.count(word) == 1:
            unique_words.append(word)
            
    filtered_list = [[word for word in sublist if word in unique_words] for sublist in words_list]
    return filtered_list

df = pd.read_csv('result_films.csv')

with open('unmeaning_words.txt', 'r') as file:
    unmeaning_words = file.read().splitlines()

# Создаем список уникальных жанров
genres = set()
for genre_list in df['genre']:
    genres.update(genre_list.split(', '))

# Обновляем значения столбцов жанров
for index, row in df.iterrows():
    genre_list = row['genre'].split(', ')
    for genre in genre_list:
        df.at[index, genre] = 1
# Извлекаем год выпуска фильма и сохраняем его в отдельный столбец "release_year"
df['release_year'] = df['release_year'].str.extract(r'(\d{4})')

# Удаляем скобки и "(I)" из столбца "release_year"
df['release_year'] = df['release_year'].str.replace(r'\(I\)', '')

# Преобразуем столбец "release_year" в числовой формат
df['release_year'] = df['release_year'].astype(int)    
    
df["imdb_rate"] = df["imdb_rate"].str.replace(',', '.').astype(float)

df_filtered = df[df['votes'] >= 100000]
# Разделение на квинтили по столбцу 'imdb_rate'
df_filtered['quintile'] = pd.qcut(df_filtered['imdb_rate'], q=5, labels=False)

# Convert 'quintile' column into binary columns
df_dummies = pd.get_dummies(df_filtered['quintile'], prefix='quintile')

# Concatenate the dummy columns with the original DataFrame
df_filtered = pd.concat([df_filtered, df_dummies], axis=1)

# Drop the original 'quintile' column
#df_filtered.drop('quintile', axis=1, inplace=True)

# Print the modified DataFrame
df_filtered = df_filtered.drop({'name', 'global_rate', 'votes', 'duration', 'genre','imdb_rate', 'quintile_0',
                                'quintile_1',  'quintile_2',  'quintile_3'}, axis=1)

# Определение признаков и целевой переменной
features = df_filtered.drop(['quintile_4'], axis=1) 
target = df_filtered['quintile_4']


# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#ищу самые популярные слова по кванттилям в тренировочной выборке
filtered_list = popular_word_list(X_train, 100)

# Создание столбцов с бинарными значениями
for i, sublist in enumerate(filtered_list):
    for word in sublist:
        column_name = f'Quan_{i}_{word}'
        X_train[column_name] = df['description'].apply(lambda x: 1 if word in x else 0)
        X_test[column_name] = df['description'].apply(lambda x: 1 if word in x else 0)



# Drop the original 'quintile' column
X_train.drop('quintile', axis=1, inplace=True)
X_test.drop('quintile', axis=1, inplace=True)
X_train.drop('description', axis=1, inplace=True)
X_test.drop('description', axis=1, inplace=True)

numeric_data = X_train.select_dtypes([np.number])

numeric_data_mean = numeric_data.mean() 
numeric_features = numeric_data.columns

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  

#нормирование данных
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

columns = X_train.columns

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)  
print(X_train)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

print('___________________')
## подбор Solver'a
## идеи с сайта https://www.kaggle.com/code/funxexcel/p2-logistic-regression-hyperparameter-tuning

logModel = LogisticRegression()
param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-5, 5, 40),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : np.logspace(1, 5, 15)
    }
]

clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1,scoring="roc_auc" )
best_clf = clf.fit(X_train_scaled,y_train)
best_clf.best_estimator_

y_pred = best_clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('максимизация roc-auc')
print('accuracy = ', accuracy, '; roc_auc = ',  roc_auc,'; precision = ', precision,'; recall = ', recall)

      release_year  metascore_rate  Adventure  Family  Fantasy  Animation  \
242           1994              63        0.0     0.0      0.0        0.0   
3191          1952              89        0.0     0.0      0.0        0.0   
2136          2008              53        0.0     0.0      0.0        0.0   
3411          1987              72        0.0     0.0      0.0        0.0   
641           2017              45        1.0     0.0      1.0        0.0   
...            ...             ...        ...     ...      ...        ...   
2515          1982              68        1.0     0.0      0.0        0.0   
1487          2013              35        0.0     0.0      0.0        0.0   
1543          2014              69        0.0     0.0      0.0        0.0   
1798          2020              63        1.0     0.0      0.0        0.0   
1126          1993              87        0.0     0.0      0.0        0.0   

      Action  Crime  Thriller  Comedy  ...  Quan_4_survive  Quan_4_island  